In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data_dir ='/kaggle/input/absa-dataset/input/sentihood/'

In [ ]:
in_file = os.path.join(data_dir, 'sentihood-train.json')

In [ ]:
def parse_sentihood_json(in_file):
    with open(in_file) as f:
        data = json.load(f)
    ret = []
    for d in data:
        text = d['text']
        sent_id = d['id']
        opinions = []
        targets = set()
        for opinion in d['opinions']:
            sentiment = opinion['sentiment']
            aspect = opinion['aspect']
            target_entity = opinion['target_entity']
            targets.add(target_entity)
            opinions.append((target_entity, aspect, sentiment))
        ret.append((sent_id, text, opinions))
    return ret

In [ ]:
import json

train = parse_sentihood_json(in_file)

In [ ]:
test_file = os.path.join(data_dir, 'sentihood-test.json')

In [ ]:
test = parse_sentihood_json(test_file)

In [ ]:
all_aspects = ['general', 'price', 'transit-location', 'safety']
def convert_input(data):
    ret = []
    for sent_id, text, opinions in data:
        for target_entity, aspect, sentiment in opinions:
            if aspect not in all_aspects:
                continue
            ret.append((sent_id, text, target_entity, aspect, sentiment))
    return ret

In [ ]:
test = convert_input(test)
test = pd.DataFrame(test)

test['id'] = test[0]
test['text'] = test[1]
test['target_entity'] = test[2]
test['label'] = test[3]
test['sentiment'] = test[4]

test = test.drop(0, axis=1)
test = test.drop(1, axis = 1)
test = test.drop(2, axis = 1)
test = test.drop(3, axis=1)
test = test.drop(4, axis=1)

In [ ]:
test.head()

In [ ]:
train = convert_input(train)
train = pd.DataFrame(train)

train['id'] = train[0]
train['text'] = train[1]
train['target_entity'] = train[2]
train['label'] = train[3]
train['sentiment'] = train[4]

train = train.drop(0, axis=1)
train = train.drop(1, axis = 1)
train = train.drop(2, axis = 1)
train = train.drop(3, axis=1)
train = train.drop(4, axis=1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.to_csv("to_train_senti.csv", index=False)
test.to_csv("to_test_senti.csv", index=False)

In [ ]:
test['text'] = test['text'] + ' ' + test['target_entity']
test.head()

In [ ]:
train['text'] = train['text'] + ' ' + train['target_entity']
#test_df['sentence1'] = test_df['sentence1'] + ' ' + test_df['sentence2'] + ' ' + test_df['label']
train.head(5)

In [ ]:
train=train.drop(["target_entity"],axis=1)


In [ ]:
test = test.drop(["target_entity"], axis=1)

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
test.groupby('label').count().plot.bar()

In [ ]:
train.groupby('label').count().plot.bar()

In [ ]:
import transformers

MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 12
EPOCHS = 10
BERT_PATH = "../input/bert-base-uncased/"
MODEL_PATH = "model.bin"
TRAINING_FILE = "../input/absa-dataset/input/bert-pair/train_NLI_M.tsv"
DEV_FILE = "../input/absa-dataset/input/bert-pair/dev_NLI_M.tsv"
TEST_FILE = "../input/absa-dataset/input/bert-pair/test_NLI_M.tsv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)


In [ ]:
import torch

class BERTDataset:
    def __init__(self, sentence1s,targets):
        self.sentence1s = sentence1s
        #self.sentence2s = sentence2s
        self.targets = targets
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        
    def __len__(self):
        return len(self.sentence1s)
                   
        
    def __getitem__(self, item):
        sentence1 = str(self.sentence1s[item])
        sentence1 = " ".join(sentence1.split())
        
        #sentence2 = str(self.sentence2s[item])
        #sentence2 = " ".join(sentence2.split())

        inputs = self.tokenizer.encode_plus(sentence1,
                                            None,
                                            add_special_tokens=True, 
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                           )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        
        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.targets[item], dtype=torch.long),
        }


In [ ]:
import transformers
import torch.nn as nn


class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 4)

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo = self.bert_drop(o2)
        output = self.out(bo)
        return output

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import pdb


def loss_fn(outputs, targets):
    #targets = torch.argmax(targets, 1)
    #loss = nn.CrossEntropyLoss()(outputs, targets.view(-1, 1))
    loss = nn.CrossEntropyLoss()(outputs, targets)
    
    return loss


def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        #pdb.set_trace()
        targets = targets.to(device, dtype=torch.long)
        #pdb.set_trace()

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.long)

            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            act = nn.Softmax(dim=1)
            outputs = act(outputs)
            fin_targets.extend(targets.detach().cpu().numpy().tolist())
            fin_outputs.extend(outputs.cpu().numpy().tolist())
            
    return fin_outputs, fin_targets

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

#from model import BERTBaseUncased
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn import preprocessing

def run():
    
    encoder = preprocessing.LabelEncoder()
    train.loc[:, "label"] = encoder.fit_transform(train["label"])
    #dev_df.loc[:, "label"] = encoder.transform(dev_df["label"])
    test.loc[:, "label"] = encoder.transform(test["label"])
    
    
    train_dataset = BERTDataset(sentence1s = train.text.values, targets=train.label.values)
    
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)
    
    valid_dataset = BERTDataset(sentence1s = test.text.values,targets=test.label.values)
    
    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=4)
    
    device = torch.device("cuda")
    model = BERTBaseUncased()
    model.to(device)
    
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [{"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": 0.001,},
                            {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0,},]
    num_train_steps = int(len(train) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    
    model = nn.DataParallel(model)
    
    best_accuracy = 0
    
    for epoch in range(EPOCHS):
        print(epoch)
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        #pdb.set_trace()
        outputs = torch.tensor(outputs)
        outputs = torch.argmax(outputs, dim=1)
        #outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score =", {accuracy})
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = accuracy
            
if __name__ == "__main__":
    run()


In [ ]:
# data1 = pd.read_csv("../input/absa-dataset/input/bert-pair/dev_NLI_M.tsv", sep='\t')

In [ ]:
# data1.head()

In [ ]:
# data1['target'] = data1['sentence2'].apply(lambda x: x.split('-')[2])
# data1['1_2'] = data1['sentence2'].apply(lambda x: x.split('-')[1])

In [ ]:
# data1['target'].head()

In [ ]:
# data1.head(-5)

In [ ]:
# def changeForFun(data1):
#     rest = []
#     for index, row in data1.iterrows():
#         response = {}
#         ids = row['id']
#         text = row['sentence1']
#         target_aspect = row['sentence2']
#         sentiment = row['label']

#         response = {
#             "opinions": [{
#                 "sentiment": sentiment,
#                 "aspect": target_aspect,
#                 "target_entity": target_aspect}
#             ],
#             "id": ids,
#             "text": text
#         }
#         rest.append(response)
#     return rest
    
    

In [ ]:
# rest = changeForFun(data1)

In [ ]:
# print(rest)